In [76]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,f1_score
os.chdir('..')
from src.utils import descriptive_statistics

# Load Data

In [77]:
file_path = "C:/Users/hp/Documents/10 academy/Week 3/AlphaCare-Insurance-Solutions/data/MachineLearningRating_v3.txt"
df = pd.read_csv(file_path,sep='|',low_memory=False)

#df = pd.read_csv(, sep='|', low_memory=False)

# Data Understanding

In [78]:
df.head(5)

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [79]:
df.tail()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
1000093,31520,389,2015-04-01 00:00:00,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000094,31520,389,2015-06-01 00:00:00,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000095,31520,389,2015-08-01 00:00:00,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000096,31519,389,2014-07-01 00:00:00,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0
1000097,31519,389,2015-02-01 00:00:00,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0


In [80]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

(1000098, 52)

In [81]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

In [82]:
df.shape


(1000098, 52)

# Descriptive Statistics

In [83]:

df['total_premium'] = df['TotalPremium']

#Descriptive statistics of TotalPremium
df['total_premium'].describe()




count    1.000098e+06
mean     6.190550e+01
std      2.302845e+02
min     -7.825768e+02
25%      0.000000e+00
50%      2.178333e+00
75%      2.192982e+01
max      6.528260e+04
Name: total_premium, dtype: float64

In [84]:
df['total_claims'] = df['TotalClaims']

#Descriptive statistics of TotalClaims
df['total_claims'].describe()

count    1.000098e+06
mean     6.486119e+01
std      2.384075e+03
min     -1.200241e+04
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.930921e+05
Name: total_claims, dtype: float64

In [85]:
#Variance

variance_premium = df['total_premium'].var()
variance_claims = df['total_claims'].var()
print(f"Variance of TotalPremium: {variance_premium}")
print(f"Variance of TotalClaims: {variance_claims}")


Variance of TotalPremium: 53030.956815882615
Variance of TotalClaims: 5683812.14896821


In [86]:
#Standard Devation
std_dev_premium = df['total_premium'].std()
std_dev_claims = df['total_claims'].std()
print(f"Standard Deviation of TotalPremium: {std_dev_premium}")
print(f"Standard Deviation of TotalClaims: {std_dev_claims}")



Standard Deviation of TotalPremium: 230.28451275733377
Standard Deviation of TotalClaims: 2384.0746945027145


In [87]:
#Categorical Variables
df.select_dtypes(include=['object'])

,TransactionMonth,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,...,CrossBorder,TermFrequency,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType
0,2015-03-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,NaN,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
1,2015-05-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,NaN,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
2,2015-07-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,NaN,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
3,2015-05-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,NaN,Monthly,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
4,2015-07-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,NaN,Monthly,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,2015-04-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,NaN,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000094,2015-06-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,NaN,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000095,2015-08-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,NaN,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000096,2014-07-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,NaN,Monthly,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant


In [88]:
df.select_dtypes(include=['float64'])

,mmcode,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,NumberOfVehiclesInFleet,SumInsured,CalculatedPremiumPerTerm,TotalPremium,TotalClaims,total_premium,total_claims
0,44069150.0,6.0,2597.0,130.0,4.0,119300.0,NaN,0.01,25.0000,21.929825,0.0,21.929825,0.0
1,44069150.0,6.0,2597.0,130.0,4.0,119300.0,NaN,0.01,25.0000,21.929825,0.0,21.929825,0.0
2,44069150.0,6.0,2597.0,130.0,4.0,119300.0,NaN,0.01,25.0000,0.000000,0.0,0.000000,0.0
3,44069150.0,6.0,2597.0,130.0,4.0,119300.0,NaN,119300.00,584.6468,512.848070,0.0,512.848070,0.0
4,44069150.0,6.0,2597.0,130.0,4.0,119300.0,NaN,119300.00,584.6468,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,4614100.0,4.0,2693.0,110.0,4.0,NaN,NaN,500000.00,395.8481,347.235175,0.0,347.235175,0.0
1000094,4614100.0,4.0,2693.0,110.0,4.0,NaN,NaN,500000.00,395.8481,347.235175,0.0,347.235175,0.0
1000095,4614100.0,4.0,2693.0,110.0,4.0,NaN,NaN,500000.00,395.8481,347.235175,0.0,347.235175,0.0
1000096,4614100.0,4.0,2693.0,110.0,4.0,NaN,NaN,5000000.00,2.6391,2.315000,0.0,2.315000,0.0


In [89]:
df.select_dtypes(include=['int64'])

,UnderwrittenCoverID,PolicyID,PostalCode,RegistrationYear
0,145249,12827,1459,2004
1,145249,12827,1459,2004
2,145249,12827,1459,2004
3,145255,12827,1459,2004
4,145255,12827,1459,2004
...,...,...,...,...
1000093,31520,389,7493,2013
1000094,31520,389,7493,2013
1000095,31520,389,7493,2013
1000096,31519,389,7493,2013


In [90]:
df.select_dtypes(include=['bool'])

,IsVATRegistered
0,True
1,True
2,True
3,True
4,True
...,...
1000093,False
1000094,False
1000095,False
1000096,False


In [91]:
df.select_dtypes(include=['bool'])
pd.CategoricalDtype

pandas.core.dtypes.dtypes.CategoricalDtype

In [92]:
df.select_dtypes(include=['bool'])

,IsVATRegistered
0,True
1,True
2,True
3,True
4,True
...,...
1000093,False
1000094,False
1000095,False
1000096,False


In [93]:
def encoder(method, dataframe, columns_label, columns_onehot):
    
    if method == 'labelEncoder':      
    
        df_lbl = dataframe.copy()
    
        for col in columns_label:
            label = LabelEncoder()
            label.fit(list(dataframe[col].values))
            df_lbl[col] = label.transform(df_lbl[col].values)
        
        return df_lbl
    
    elif method == 'oneHotEncoder':
        
        df_oh = dataframe.copy()

        df_oh= pd.get_dummies(data = df_oh, prefix = 'ohe', prefix_sep='_',
                       columns = columns_onehot,
                       drop_first =True,
                       dtype='int8')
        
        return df_oh




In [94]:
method =['labelEncoder', 'oneHotEncoder']
dataframe = df.copy()
columns_label = ['IsVATRegistered', 'Title', 'Gender']
columns_onehot = ['IsVATRegistered', 'Title', 'Gender', 'MaritalStatus']

df_label = encoder(method[0], dataframe, columns_label, columns_onehot)

df_onehot = encoder(method[1], dataframe, columns_label, columns_onehot)

df_label.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,total_premium,total_claims
0,145249,12827,2015-03-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0


In [95]:
df.info()
df.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 54 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

(1000098, 54)

In [96]:
df['transaction_month']=df['TransactionMonth']
df['transaction_month'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

In [97]:
df.head(5)

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,total_premium,total_claims,transaction_month
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0,2015-03-01
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0,2015-05-01
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0,2015-07-01
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,512.848070,0.0,2015-05-01
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0,2015-07-01


In [106]:
def scaler(method, data, columns_scaler):
        if method == 'standartScaler':
            Standard = StandardScaler()
            df_standard = data.copy()
            df_standard[columns_scaler]=Standard.fit_transform(df_standard[columns_scaler])        
            return df_standard
        
        elif method == 'minMaxScaler':        
            MinMax= MinMaxScaler()
            df_minmax = data.copy()
            df_minmax[columns_scaler]=MinMax.fit_transform(df_minmax[columns_scaler])        
            return df_minmax
    
        elif method =='npLog':        
            df_nplog = data.copy()
            df_nplog[columns_scaler]=np.log(df_nplog[columns_scaler])        
            return df_nplog
    
        elif method == 'default':

            return data


In [107]:
method = 'minMaxScaler'
data = df_label
columns_scaler = ['TotalPremium']
df_scaler = scaler(method, data, columns_scaler)


In [108]:
df_scaler.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,total_premium,total_claims
0,145249,12827,2015-03-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.012177,0.0,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.012177,0.0,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.011846,0.0,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.019608,0.0,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,1,,Close Corporation,2,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.011846,0.0,0.000000,0.0
